In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
import scipy.optimize

In [2]:
def softmax_cost(theta, n_classes, input_size, lambda_, data, labels):
    """
    Compute the cost and derivative.
    n_classes: the number of classes
    input_size: the size N of the input vector
    lambda_: weight decay parameter
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    labels: an M x 1 matrix containing the labels corresponding for the input data
    """
    # k stands for the number of classes
    # n is the number of features and m is the number of samples
    k = n_classes
    n, m = data.shape
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Matrix of indicator fuction with shape (k, m)
    labels = np.reshape(labels,(m,))
    indicator = scipy.sparse.csr_matrix((np.ones(m), (labels, np.arange(m))))
    indicator = np.array(indicator.todense())
    # Cost function
    cost = -1.0/m * np.sum(indicator * np.log(proba)) + 0.5*lambda_*np.sum(theta*theta)
    # Gradient matrix with shape (k, n)
    grad = -1.0/m * (indicator - proba).dot(data.T) + lambda_*theta
    # Unroll the gradient matrices into a vector
    grad = grad.ravel()
    return grad

Define a gradient descent method to obtain optimum parameter values. 

In [3]:
def gdm(theta, n_classes, input_size, lambda_,data,labels):
    #norm= np.linalg.norm(thetaOld - theta,ord= 2)
    #print "norm", norm
    thetaOld= theta*9999
    errTol=0.001
    while np.linalg.norm(thetaOld - theta,ord= 2) > errTol:
        thetaOld= theta
        theta= thetaOld - lambda_ * softmax_cost(theta, n_classes, input_size, lambda_, data, labels)
        norm= np.linalg.norm(thetaOld - theta,ord=2)
        #print "norm",norm,
    return theta

In [4]:
def softmax_train(input_size, n_classes, lambda_, input_data, labels, options={'maxiter': 400, 'disp': True}):
    """
    Train a softmax model with the given parameters on the given data.
    Returns optimal theta, a vector containing the trained parameters
    for the model.
    input_size: the size of an input vector x^(i)
    n_classes: the number of classes
    lambda_: weight decay parameter
    input_data: an N by M matrix containing the input data, such that
                input_data[:, c] is the c-th input
    labels: M by 1 matrix containing the class labels for the
            corresponding inputs. labels[c] is the class label for
            the c-th input
    options (optional): options
      options['maxiter']: number of iterations to train for
    """
    # initialize parameters
    theta = 0.005 * np.random.randn(n_classes * input_size)
    #J = lambda theta : softmax_cost(theta, n_classes, input_size, lambda_, input_data, labels)
    # Find out the optimal theta
    #results = scipy.optimize.minimize(J, theta, method='L-BFGS-B', jac=True, options=options)
    #opt_theta = results['x']
    opt_theta= gdm(theta, n_classes, input_size, lambda_, input_data, labels)
    model = {'opt_theta': opt_theta, 'n_classes': n_classes, 'input_size': input_size}
    return model

In [5]:
def softmax_predict(model, data):
    """
    model: model trained using softmax_train
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    pred: the prediction array.
    """
    theta = model['opt_theta'] # Optimal theta
    k = model['n_classes']  # Number of classes
    n = model['input_size'] # Input size (number of features)
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Prediction values
    pred = np.argmax(proba, axis=0)
    return pred


Upload train data, train label, test data  and test label. I assume that we are working on mnist dataset.

In [ ]:
X = load(tainDAta)
y = load(testLabels)
X_test = load(testData)
y_test = load(testLabels)

In [ ]:
input_size = 28 * 28 # Size of input vector (MNIST images are 28x28)
n_classes = 10       # Number of classes (MNIST images fall into 10 classes)
lambda_ = 0.5 # Weight decay parameter

In [ ]:
options = {'maxiter': 100, 'disp': True}
model = softmax_train(input_size, n_classes, lambda_, X, y, options)

In [ ]:
pred = softmax_predict(model, X_test)
y_test = y_test.tolist()
pred = pred.tolist()
print accuracy_score(y_test,pred)

# PERFORMANCE ANALYSIS

|Error tolerance|learning rate|Accuracy|
|---------------|-------------|--------|
|     0.002     |   0.001     | 85     |
|     0.002     |    0.05     | 87     |
|     0.002     |    0.1      |  87.29 |
|     0.001     |    0.5      |  86.79 |
|     0.001     |    0.1      |  86.79 |

From above analysis we can conclude that this model gives accuracy of 87.29 % with parameters set to error tolerance = 0.002 and step size = 0.1. 